# Notebook to test the Bedrock Agents and UC Functions.

In [23]:
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.client import ApiClient, Configuration


In [24]:
from unitycatalog.ai.bedrock.toolkit import UCFunctionToolkit

In [25]:
config = Configuration()
config.host = "http://localhost:8080/api/2.1/unity-catalog"

# The base ApiClient is async
api_client = ApiClient(configuration=config)

client = UnitycatalogFunctionClient(api_client=api_client)

CATALOG = "AICatalog"
SCHEMA = "AISchema"


In [28]:
# Sample function
    
def location_weather_in_c(location_id: str, fetch_date: str) -> str:
    """Test function for AWS Bedrock integration.

    Args:
        location_id (str): The name to be included in the greeting message.
        fetch_date (str): The date with the location 

    Raises:
        Exception: If there is an error during the function execution.

    Returns:
        str: Weather result.
    """
    try:
        # Fetch from Databricks SQL Warehouse based UC function execution 
        return "23"
    except Exception as e:
        raise Exception(f"Error occurred: {e}")

In [29]:
client.uc.create_catalog(name=CATALOG, comment="Catalog for AI functions")

CatalogInfo(name='AICatalog', comment='Catalog for AI functions', properties={}, owner=None, created_at=1737005234233, created_by=None, updated_at=1737005234233, updated_by=None, id='7dc1e000-2854-46ac-9811-d140c1cb942e')

In [30]:
client.uc.create_schema(catalog_name=CATALOG, name=SCHEMA, comment="Schema for AI functions")

SchemaInfo(name='AISchema', catalog_name='AICatalog', comment='Schema for AI functions', properties={}, full_name='AICatalog.AISchema', owner=None, created_at=1737005278533, created_by=None, updated_at=1737005278533, updated_by=None, schema_id='c164dc56-8ee1-489c-9dad-19734bbcb0d7')

In [32]:
client.create_python_function(
    func=location_weather_in_c, 
    catalog=CATALOG, 
    schema=SCHEMA, 
    replace=True)

FunctionInfo(name='location_weather_in_c', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='location_id', type_text='STRING', type_json='{"name": "location_id", "type": "string", "nullable": false, "metadata": {"comment": "The name to be included in the greeting message."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The name to be included in the greeting message.'), FunctionParameterInfo(name='fetch_date', type_text='STRING', type_json='{"name": "fetch_date", "type": "string", "nullable": false, "metadata": {"comment": "The date with the location"}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=1, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The date with t

In [33]:
client.list_functions(catalog=CATALOG, schema=SCHEMA)

[FunctionInfo(name='bedrock_test_function', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='name', type_text='STRING', type_json='{"name": "name", "type": "string", "nullable": false, "metadata": {"comment": "The name to be included in the greeting message."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The name to be included in the greeting message.')]), data_type=<ColumnTypeName.STRING: 'STRING'>, full_data_type='STRING', return_params=None, routine_body='EXTERNAL', routine_definition='try:\n    # Fetch from Databricks SQL Warehouse based UC function execution \n    return "hello: " + name\nexcept Exception as e:\n    raise Exception(f"Error occurred: {e}")', routine_dependencies=None, parameter_style='S', is_deterministic=True, sql_data_access='NO_SQL', i

In [34]:
function_name = f"{CATALOG}.{SCHEMA}.location_weather_in_c"
toolkit = UCFunctionToolkit(function_names=[function_name], client=client)

In [35]:
print(toolkit)

function_names=['AICatalog.AISchema.location_weather_in_c'] tools_dict={'AICatalog.AISchema.location_weather_in_c': BedrockTool(name='AICatalog__AISchema__location_weather_in_c', description='Test function for AWS Bedrock integration.', parameters={'type': 'object', 'properties': {'location_id': {'default': None, 'description': 'The name to be included in the greeting message.', 'title': 'Location Id', 'type': 'string'}, 'fetch_date': {'default': None, 'description': 'The date with the location', 'title': 'Fetch Date', 'type': 'string'}}, 'required': []}, requireConfirmation='ENABLED')} client=<unitycatalog.ai.core.client.UnitycatalogFunctionClient object at 0x1227e1190>


# Adding Bedrock SDK calls 


In [36]:
import boto3, pprint, json, time, uuid

In [37]:
boto3.setup_default_session()

In [38]:
agent_id = "AP5RQUVNTU"
agent_alias_id = "O6EXN8DJVZ"

In [39]:
# Bedrock agent configuration
session = toolkit.create_session(agent_id=agent_id,
                                agent_alias_id=agent_alias_id)
# Generate unique session ID
session_id = str(uuid.uuid1())

In [40]:
response = session.invoke_agent(
                input_text="What is the weather for location 1234 and date of 2024-11-19",
                enable_trace=True,
                session_id=session_id)

In [41]:
import pprint 
pprint.pp(response)

{'ResponseMetadata': {'RequestId': 'e6a6fac9-fb25-4277-b517-390b84b82227',
                      'HTTPStatusCode': 200,
                      'HTTPHeaders': {'date': 'Mon, 20 Jan 2025 19:09:43 GMT',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'transfer-encoding': 'chunked',
                                      'connection': 'keep-alive',
                                      'x-amzn-requestid': 'e6a6fac9-fb25-4277-b517-390b84b82227',
                                      'x-amz-bedrock-agent-session-id': '18915a4e-d762-11ef-aef3-4e3e215d14d1',
                                      'x-amzn-bedrock-agent-content-type': 'application/json'},
                      'RetryAttempts': 0},
 'contentType': 'application/json',
 'sessionId': '18915a4e-d762-11ef-aef3-4e3e215d14d1',
 'completion': <botocore.eventstream.EventStream object at 0x126841ac0>}


In [42]:
event_stream = response['completion']

for event in event_stream:
   pprint.pp(event)

{'trace': {'agentAliasId': 'O6EXN8DJVZ',
           'agentId': 'AP5RQUVNTU',
           'agentVersion': '1',
           'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-1:288584671716:agent-alias/AP5RQUVNTU/O6EXN8DJVZ'}],
           'sessionId': '18915a4e-d762-11ef-aef3-4e3e215d14d1',
           'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048,
                                                                                                'stopSequences': ['</invoke>',
                                                                                                                  '</answer>',
                                                                                                                  '</error>'],
                                                                                                'temperature': 0.0,
                                                                                        

In [44]:

function_input = event["returnControl"]["invocationInputs"][0]["functionInvocationInput"]

print(f"Function to call: {function_input['function']}")

print(f"Parameters: {function_input['parameters']}")

# Simulate weather result (replace with actual API call)
weather_result = "23"  # Example temperature
# Send result back to agent
final_response = session.invoke_agent(
    input_text="",
    session_id=session_id,
    enable_trace=True,
    session_state={
        'invocationId': event["returnControl"]["invocationId"],
        'returnControlInvocationResults': [{
            'functionResult': {
                'actionGroup': function_input["actionGroup"],
                'function': function_input["function"],
                'confirmationState': 'CONFIRM',
                'responseBody': {
                    "TEXT": {
                        'body':
                        f"weather_in_centigrade: {weather_result}"
                    }
                }
            }
        }]
    }
)



Function to call: location_weather_in_c
Parameters: [{'name': 'fetch_date', 'type': 'string', 'value': '2024-11-19'}, {'name': 'location_id', 'type': 'integer', 'value': '1234'}]


In [45]:
# Print final response
print("Agent Response:")
for final_event in final_response.get('completion', []):
    print(f"  {final_event}")

Agent Response:
  {'trace': {'agentAliasId': 'O6EXN8DJVZ', 'agentId': 'AP5RQUVNTU', 'agentVersion': '1', 'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-1:288584671716:agent-alias/AP5RQUVNTU/O6EXN8DJVZ'}], 'sessionId': '18915a4e-d762-11ef-aef3-4e3e215d14d1', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048, 'stopSequences': ['</invoke>', '</answer>', '</error>'], 'temperature': 0.0, 'topK': 250, 'topP': 1.0}, 'text': '{"system":"You are a weather agent to fetche the current weather in celsius for a given locationYou have been provided with a set of functions to answer the user\'s question.You will ALWAYS follow the below guidelines when you are answering a question:<guidelines>- Think through the user\'s question, extract all data from the question and the previous conversations before creating a plan.- ALWAYS optimize the plan by using multiple function calls at the same time whenever possible.- Never assume any para

# Function Execution

## Listing Function

In [49]:
functions = client.list_functions(
    catalog=CATALOG,
    schema=SCHEMA,
    max_results=10  # Paginated results will contain a continuation token that can be submitted with additional requests
)

print(functions)

[FunctionInfo(name='bedrock_test_function', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='name', type_text='STRING', type_json='{"name": "name", "type": "string", "nullable": false, "metadata": {"comment": "The name to be included in the greeting message."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The name to be included in the greeting message.')]), data_type=<ColumnTypeName.STRING: 'STRING'>, full_data_type='STRING', return_params=None, routine_body='EXTERNAL', routine_definition='try:\n    # Fetch from Databricks SQL Warehouse based UC function execution \n    return "hello: " + name\nexcept Exception as e:\n    raise Exception(f"Error occurred: {e}")', routine_dependencies=None, parameter_style='S', is_deterministic=True, sql_data_access='NO_SQL', i

In [63]:
for f in functions:
    if f.name == "location_weather_in_c":
      print(f"Function: {f.name}")
      print(f"Parameters: {f.input_params}")

Function: location_weather_in_c
Parameters: parameters=[FunctionParameterInfo(name='location_id', type_text='STRING', type_json='{"name": "location_id", "type": "string", "nullable": false, "metadata": {"comment": "The name to be included in the greeting message."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The name to be included in the greeting message.'), FunctionParameterInfo(name='fetch_date', type_text='STRING', type_json='{"name": "fetch_date", "type": "string", "nullable": false, "metadata": {"comment": "The date with the location"}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=1, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The date with the location')]


## Executing Function

In [74]:

location_id = "New Jersey"
fetch_date = "2025-01-01"

parameters = {"location_id": "New Jersey", "fetch_date": "2025-01-01"}
result = client.execute_function(function_name, parameters)


In [77]:
value =  result.value
print(f"The weather in centigrade for {location_id} on {fetch_date} is: {value}")

The weather in centigrade for New Jersey on 2025-01-01 is: 23
